In [13]:
from scipy import linalg, mat, dot
import numpy as np
import scipy as sp
from numpy.random import random
import os
import numpy as np
import pandas as pd
import tqdm
import time
import math
from tqdm import tqdm_notebook


dir_ = '../data_0.5_track_20/'
# file_name = 'normalized_filter_track_5_user_100.csv'
file_name = 'normalized_to_rating_filter_track_20_user_100.csv'

In [14]:
# B = [[3,7,5,1],[5,3,8,3],[2,4,6,9]]
# A = mat(test_matrix) 
# U, s, V = linalg.svd(A) 
# S = linalg.diagsvd(s,len(U),len(V)) 
# RA = dot(dot(U,S),V) 
#------------------------------------------
# from scipy.sparse import csc_matrix
# from scipy.sparse.linalg import svds
# B = [[1,1,4,5],
#      [1,0,5,4],
#      [5,4,0,1],
#      [4,5,1,1]]
# C = mat(B, dtype=float)
# u, s, vt = svds(C, k=40) # k is the number of factors

In [15]:
train = np.loadtxt(os.path.join(dir_, 'rating_matrix_' + file_name), delimiter=',')
pd_train = pd.read_pickle(os.path.join(dir_, 'train_' + file_name[:-3] + 'pkl'))
pd_test = pd.read_pickle(os.path.join(dir_, 'test_' + file_name[:-3] + 'pkl'))

In [16]:
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import svds
M = csc_matrix(train, dtype=float)
k = 100
u, s, vt = svds(M, k=k) # k is the number of factors
S = linalg.diagsvd(s,k,k) 

In [17]:
print(len(u), len(u[0]))
print(len(vt), len(vt[0]))
print(len(S))
# np.set_printoptions(suppress=True)
# print(u, '\n\n', S, '\n\n', vt)
# RA = dot(dot(u,S),vt) 
# RA
# np.set_printoptions(suppress=True)
# print(train)
# print(len(train), len(train[0]))

211 100
100 8941
100


In [18]:
def pearson_correlation(u1_index, u2_index):
        result = 0.0
        user1_data = u[u1_index]
        user2_data = u[u2_index]

        rx_avg = user_average_rating(user1_data)
        ry_avg = user_average_rating(user2_data)

        top_result = 0.0
        bottom_left_result = 0.0
        bottom_right_result = 0.0
        for item in range(len(user1_data)):
            rxs = user1_data[item]
            rys = user2_data[item]

            top_result += (rxs - rx_avg)*(rys - ry_avg)
            bottom_left_result += pow((rxs - rx_avg), 2)
            bottom_right_result += pow((rys - ry_avg), 2)
        bottom_left_result = math.sqrt(bottom_left_result)
        bottom_right_result = math.sqrt(bottom_right_result)
        
        ################################################################
        if (bottom_left_result * bottom_right_result) == 0:
            return -2 # dump the data
        ################################################################
        
        result = top_result/(bottom_left_result * bottom_right_result)
        return result

def user_average_rating(u):
    avg_rating = 0.0
    for i in u:
        avg_rating += i
    avg_rating /= len(u) * 1.0
    return avg_rating

In [19]:
def knn(uid, k):
    neighbors = []
    result = []
    for user in range(len(train)):
        if uid == user:
            continue
        upc = pearson_correlation(uid, user)
        neighbors.append([user, upc])
    sorted_neighbors = sorted(neighbors, key=lambda neighbors: (neighbors[1], neighbors[0]), reverse=True)   # - for desc sort

    for i in range(k):
        if i >= len(sorted_neighbors):
            break
        result.append(sorted_neighbors[i])
    return result

In [20]:
def predict_by_knn(uid, tid, k_nearest_neighbors):
        valid_neighbors = check_neighbors_validattion(tid, k_nearest_neighbors)
        ##################################
        if not len(valid_neighbors):
            return -1
        ##################################
        top_result = 0.0
        bottom_result = 0.0
        for neighbor in valid_neighbors:
            neighbor_id = neighbor[0]
            neighbor_similarity = neighbor[1]   # Wi1
            rating = train[neighbor_id][tid] # rating i,item
            top_result += neighbor_similarity * rating
            bottom_result += neighbor_similarity
        result = top_result/bottom_result
        return result

def check_neighbors_validattion(tid, k_nearest_neighbors):
    result = []
    for neighbor in k_nearest_neighbors:
        neighbor_id = neighbor[0]
        if train[neighbor_id][tid] > 0:
            result.append(neighbor)
    return result


In [21]:
def predict(uid, tid, k):
    
    if len(knn_list[uid]) == 0:
        k_nearest_neighbors = knn(uid, k)
        knn_list[uid] = k_nearest_neighbors
    else:
        k_nearest_neighbors = knn_list[uid]

    prediction = predict_by_knn(uid, tid, k_nearest_neighbors)
    return prediction

In [22]:
knn_list = []

for i in range(len(train)):
    knn_list.append([])

In [23]:
num_users = len(train)
num_tracks = len(train[0])
print(num_users,num_tracks)

211 8941


In [24]:
k = 30
pd_prediction = pd.DataFrame(columns=['uid','tid','rating'])

for i in tqdm.tqdm_notebook(range(num_users)):
    
    user = pd_train[pd_train['uid']==i]
    target_user = i
    
    for j in range(num_tracks):
        r = user[user['tid']==j]['rating']
        if len(r) == 0:
            target_item = j
            prediction = predict(target_user, target_item, k)
        else:
            prediction = -1
        if not prediction == -1:
            pd_prediction = pd_prediction.append({'uid': i, 'tid': j, 'rating': prediction}, ignore_index=True)

In [25]:
pd_prediction

,uid,tid,rating
0,0.0,0.0,3.049288
1,0.0,1.0,3.189951
2,0.0,3.0,1.738379
3,0.0,4.0,3.214958
4,0.0,13.0,1.350822
...,...,...,...
1089534,210.0,8931.0,0.250000
1089535,210.0,8934.0,3.250000
1089536,210.0,8935.0,0.250000
1089537,210.0,8937.0,3.761703


In [26]:
pd_prediction.to_pickle(os.path.join(dir_, 'prediction_SVD_user_top_N_' + file_name[:-3] + 'pkl'))

In [27]:
knn_list

[[[124, 0.27633849413426315],
  [36, 0.20049700740051238],
  [206, 0.19428947839904548],
  [148, 0.18493334756249985],
  [160, 0.17468025668444556],
  [116, 0.1743223034593887],
  [191, 0.1662056430059415],
  [64, 0.16012405200707408],
  [95, 0.1532516330382832],
  [141, 0.15235540051826735],
  [179, 0.14370626839393452],
  [155, 0.1392703189476304],
  [208, 0.13711333767746817],
  [2, 0.11742636697215907],
  [89, 0.11446415649606467],
  [136, 0.113207927756727],
  [55, 0.11307613458723025],
  [107, 0.1117293624722347],
  [182, 0.09821329298208702],
  [50, 0.09236329585150138],
  [43, 0.07914870859110086],
  [48, 0.07451719681701344],
  [130, 0.06452173223520898],
  [109, 0.06184229508256528],
  [53, 0.060577320377229696],
  [32, 0.05669098541416364],
  [21, 0.05293487097838566],
  [184, 0.04891657857353343],
  [102, 0.04787112621146913],
  [61, 0.04721811742432277]],
 [[161, 0.5326976552362588],
  [47, 0.5140229995065385],
  [199, 0.4310853352982136],
  [5, 0.40519990271692935],
  [44

In [28]:
k = knn_list

In [29]:
pd_knn = pd.DataFrame(columns=['uid','knn_uid','similarity'])
n=0
for i in knn_list:
    for j in i:
        pd_knn = pd_knn.append({'uid': n, 'knn_uid': j[0], 'similarity': j[1]}, ignore_index=True)
    n += 1

In [30]:
pd_knn.to_pickle(os.path.join(dir_, 'knn_SVD_user_top_N_' + file_name[:-3] + 'pkl'))

In [38]:
n

220